In [1]:
import numpy as np 
import matplotlib.pyplot as plt 
import sklearn.neighbors
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler

import pytesseract
from PIL import Image
import cv2
import pandas as pd 
import skimage
from pathlib import Path

## Dataset paths and loading

In [2]:
from common import *

## Load sample image

In [3]:
image_id = 2
im, data = load_sample(image_id)

## Drawing funcs

In [4]:
# Linetypes
def draw_pipelines(image, data=data):
    draw = image.copy()
    solid_lines = np.stack(data["lines"].query("type=='solid'")["box"])
    dashed_lines = np.stack(data["lines"].query("type=='dashed'")["box"])

    draw = cv2.drawContours(draw, solid_lines.reshape(-1,2,2), -1, (255, 255, 0), thickness=2)
    draw = cv2.drawContours(draw, dashed_lines.reshape(-1,2,2), -1, (0, 255, 255), thickness=2)
    return draw

def draw_detections(image, rects, classes,color=(255,0,0)):
    draw_rects(image, rects, thickness=8, color=color)
    for r,c in zip(rects,classes):
        cv2.putText(image, str(c), r.flatten()[:2], cv2.FONT_HERSHEY_PLAIN, 4, (0,0,0))

def draw_symbols(image, data=data, color=None, thickness=2):
    draw = image.copy()
    for i, group in data["symbols"].groupby("class"):
        color_ = color or (np.random.rand(3)*255).astype(np.uint8)
        symbols = np.stack(group["box"])
        draw_rects(draw, symbols, color=[int(c) for c in color_], thickness=thickness)
    return draw

def draw_gt_symbols(image, data=data,color=None, thickness=2):
    symbol_boxes = np.stack(data["symbols"]["box"])
    symbol_classes = np.stack(data["symbols"]["class"]).astype(int)
    draw_detections(image, symbol_boxes, symbol_classes, color=color)

def draw_text_boxes(image, data=data, color=(255,0,255), thickness=1):
    draw = image.copy()
    text_boxes = np.stack(data["words"]["box"])
    draw_rects(draw, text_boxes, color=color, thickness=thickness)
    return draw


In [5]:
%matplotlib tk


# im = cv2.imread("test.jpg")
draw = im.copy()
draw = draw_pipelines(draw)
draw = draw_symbols(draw)
draw = draw_text_boxes(draw)
plt.imshow(draw)

## Detection with blackhat

In [49]:
def detect_symbols(image):

    if image.ndim == 3:
        gray = np.mean(image,axis=-1).astype(np.uint8)
    else:
        gray=image

    t, thresh = cv2.threshold(gray, 0, 255, cv2.THRESH_BINARY | cv2.THRESH_OTSU)

    # Foreground is smaller than 50% of image
    if np.count_nonzero(thresh) > thresh.size/2:
        thresh = 255-thresh

    skel = skimage.morphology.skeletonize(thresh//255, method="lee")

    kern = cv2.getStructuringElement(cv2.MORPH_RECT, ksize=(35,35))
    closing_kern = cv2.getStructuringElement(cv2.MORPH_RECT, ksize=(5,5))

    blackhat = cv2.morphologyEx(skel, cv2.MORPH_BLACKHAT, kern)

    blackhat = cv2.morphologyEx(blackhat, cv2.MORPH_OPEN, closing_kern)
    blackhat = cv2.morphologyEx(blackhat, cv2.MORPH_CLOSE, closing_kern, iterations=3)

    blackhat = cv2.morphologyEx(blackhat, cv2.MORPH_DILATE, closing_kern, iterations=2)
    blackhat = cv2.morphologyEx(blackhat, cv2.MORPH_ERODE, closing_kern, iterations=1)

    contours, hierarchy = cv2.findContours(blackhat*255, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)

    symbol_boxes = []
    for c in contours:
        if cv2.contourArea(c) > 10:
            x,y,w,h =cv2.boundingRect(c)
            symbol_boxes.append([x,y,x+w,y+h])

    boxes = np.stack(symbol_boxes)
    return non_max_suppression_fast(boxes,overlapThresh=0.4)


In [37]:
def detect_symbols_2(image):
    if image.ndim == 3:
        gray = np.mean(image,axis=-1).astype(np.uint8)
    else:
        gray=image

    t, thresh = cv2.threshold(gray, 0, 255, cv2.THRESH_BINARY | cv2.THRESH_OTSU)

    # Foreground is smaller than 50% of image
    if np.count_nonzero(thresh) > thresh.size/2:
        thresh = 255-thresh

    skel = skimage.morphology.skeletonize(thresh//255, method="lee")

    vkern = cv2.getStructuringElement(cv2.MORPH_RECT, ksize=(1,35))
    hkern = cv2.getStructuringElement(cv2.MORPH_RECT, ksize=(35,1))
    closing_kern = cv2.getStructuringElement(cv2.MORPH_RECT, ksize=(3,3))
    # kern = cv2.getStructuringElement(cv2.MORPH_RECT, ksize=(5,5))

    # blackhat = cv2.morphologyEx(skel, cv2.MORPH_DILATE, closing_kern)
    blackhat=skel

    verod = cv2.morphologyEx(blackhat, cv2.MORPH_ERODE, vkern)
    herod = cv2.morphologyEx(blackhat, cv2.MORPH_ERODE, hkern)

    blackhat = verod | herod

    blackhat = skel - blackhat

    blackhat = cv2.morphologyEx(blackhat, cv2.MORPH_DILATE, closing_kern)

    contours, hierarchy = cv2.findContours(blackhat*255, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)

    symbol_boxes = []
    for c in contours:
        if cv2.contourArea(c) > 150:
            x,y,w,h =cv2.boundingRect(c)
            symbol_boxes.append([x,y,x+w,y+h])

    boxes = np.stack(symbol_boxes)
    return non_max_suppression_fast(boxes,overlapThresh=0.4)

## Detect without text removal

In [7]:
fig, axs = plt.subplots(1,2, sharex=True, sharey=True)

draw = im.copy()

symbol_boxes = detect_symbols(im)

draw_rects(draw, np.stack(symbol_boxes).reshape(-1,2,2), thickness=8)
axs[0].imshow(draw)

draw2 = draw_symbols(draw, thickness=8, color=(0,255,0))
axs[1].imshow(draw2)

## Detect with text removal

In [39]:
fig, axs = plt.subplots(1,2, sharex=True, sharey=True)

# Pre-process
im_cleanup = cleanup_text(im, img_id=image_id)
draw = im_cleanup.copy()

# Detect
symbol_boxes = detect_symbols(im_cleanup)

# Draw results
draw_rects(draw, np.stack(symbol_boxes).reshape(-1,2,2), thickness=8)
axs[0].imshow(draw)
draw2 = draw_symbols(draw, thickness=8, color=(0,255,0))
axs[1].imshow(draw2)

KeyboardInterrupt: 

### Detect with gt text

In [8]:
clean = im.copy()

clean = draw_text_boxes(clean, color=(255,255,255), thickness=-1)
plt.imshow(clean)

In [51]:
fig, axs = plt.subplots(1,2, sharex=True, sharey=True)

# Pre-process
im_cleanup = clean
draw = im_cleanup.copy()

# Detect
symbol_boxes = detect_symbols(im_cleanup)

# Draw results
draw_rects(draw, np.stack(symbol_boxes).reshape(-1,2,2), thickness=8)
axs[0].imshow(draw)
axs[0].set_title("Predicted")
axs[0].axis("off")
draw2 = draw_symbols(draw, thickness=8, color=(0,255,0))
axs[1].imshow(draw2)
axs[1].set_title("+Ground truth")
axs[1].axis("off")

precision, recall = detection_metrics(symbol_boxes, np.stack(data["symbols"].box))

print(f"{precision=}\n{recall=}")

precision=0.5197368421052632
recall=0.7117117117117117


In [38]:
fig, axs = plt.subplots(1,2, sharex=True, sharey=True)

# Pre-process
im_cleanup = clean
draw = im_cleanup.copy()

# Detect
symbol_boxes = detect_symbols_2(im_cleanup)

# Draw results
draw_rects(draw, np.stack(symbol_boxes).reshape(-1,2,2), thickness=8)
axs[0].imshow(draw)
draw2 = draw_symbols(draw, thickness=8, color=(0,255,0))
axs[1].imshow(draw2)

precision, recall = detection_metrics(symbol_boxes, np.stack(data["symbols"].box))

print(f"{precision=}\n{recall=}")

precision=0.6229508196721312
recall=0.6846846846846847


## Classsification

### Features for each object

In [ ]:
import mahotas 

def get_largest_contour(im):
    contours, hierarchy = cv2.findContours(im, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)
    cmax = sorted(contours, key=cv2.contourArea)[-1]
    return cmax

def zernike_adaptive_centroid(image, degree=8):
    c = get_largest_contour(image)
    (x,y),r = cv2.minEnclosingCircle(c)
    return  mahotas.features.zernike_moments(image, r, degree=degree)

def rect_to_slice(rect_pts, margin=0):
    """
    Convert cv-style rect to numpy-style slice
    """
    (x0, y0), (x1, y1) = rect_pts

    return (slice(y0-margin, y1+margin), slice(x0-margin, x1+margin))

In [ ]:
def get_square_thresh(image):
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    t, thresh = cv2.threshold(gray, 0, 255, cv2.THRESH_BINARY_INV | cv2.THRESH_OTSU)

    return resizeAndPad(thresh, (64,64), padColor=0)

def get_zernike_features(image):
    thresh = get_square_thresh(image)
    return zernike_adaptive_centroid(thresh)

crops = [ im_cleanup[rect_to_slice(s.reshape(2,2), margin=15)] for s in symbol_boxes] 
features = [ get_zernike_features(crop) for crop in crops]

print(np.stack(features))

#### Clustering

In [ ]:
import sklearn.cluster

centroid, labels, _ = sklearn.cluster.k_means(np.stack(features), n_clusters=5)
# b = sklearn.cluster.estimate_bandwidth(np.stack(features))
# centroid, labels = sklearn.cluster.mean_shift(np.stack(features), bandwidth=b)

In [ ]:
for l in np.unique(labels):
    fig,ax = plt.subplots(1, 1+np.count_nonzero(labels==l))
    i=0
    for label, crop in zip(labels,crops):
        if label==l:
            ax[i].imshow(crop)
            i+=1

### Training split

In [ ]:
import re

def load_training_set(feature_func):
    training_img_ids = range(400,500) 
    templates_path = Path("../templates")

    file_pattern = re.compile(r"im(\d+)_sym(\d+)")

    train_x = []
    train_y = []
    for template in templates_path.glob("**/*.png"):
        file = template.stem
        id, sym = file_pattern.match(file).groups() 

        if int(id) in training_img_ids:
            template_im = cv2.imread(str(template))

            # Features
            train_x.append(feature_func(template_im))
            # Folder is the class
            train_y.append(int(template.parent.stem))

    return np.stack(train_x), np.stack(train_y)


In [68]:
def eval_results(image, boxes, feature_func, class_pipeline):
    fig, axs = plt.subplots(1,2, sharex=True, sharey=True)

    draw = image.copy()

    # Detect
    # symbol_boxes = detect_symbols(im_cleanup)
    # symbol_boxes = np.stack(data["symbols"]["box"])
    crops = [ image[rect_to_slice(s.reshape(2,2), margin=15)] for s in boxes] 
    features = [ feature_func(crop) for crop in crops]

    probas = class_pipeline.predict_proba(features)
    predictions = 1+np.argmax(probas,axis=1)
    confidences = np.max(probas,axis=1)

    draw_gt_symbols(draw2, thickness=8, color=(0,255,0))
    axs[0].imshow(draw2)

    # Draw results
    draw_detections(draw, np.stack(boxes).reshape(-1,2,2), predictions)
    axs[1].imshow(draw)

In [62]:
pipe = Pipeline([("scaler", StandardScaler()), ("classifier", sklearn.neighbors.KNeighborsClassifier(n_neighbors=5))])

train_x, train_y = load_training_set(feature_func=get_zernike_features)
pipe.fit(train_x, train_y)

Pipeline(steps=[('scaler', StandardScaler()),
                ('classifier', KNeighborsClassifier())])

In [71]:
eval_results(image=im_cleanup, 
             boxes=detect_symbols(im_cleanup),
             feature_func=get_zernike_features,
             class_pipeline=pipe)

In [73]:
pipe = Pipeline([("classifier", sklearn.neighbors.KNeighborsClassifier(n_neighbors=5, metric="hamming"),)])

ham_features = lambda x: get_square_thresh(x).ravel()
train_x, train_y = load_training_set(feature_func=ham_features)
pipe.fit(train_x, train_y)

Pipeline(steps=[('classifier', KNeighborsClassifier(metric='hamming'))])

In [74]:
eval_results(image=im_cleanup, 
             boxes=detect_symbols(im_cleanup),
             feature_func=ham_features,
             class_pipeline=pipe)

In [ ]:

fig, axs = plt.subplots(1,2, sharex=True, sharey=True)

draw = im_cleanup.copy()

# Detect
# symbol_boxes = detect_symbols(im_cleanup)
symbol_boxes = np.stack(data["symbols"]["box"])
predictions = np.stack(data["symbols"]["class"]).astype(int)

# Draw results
draw_detections(draw, np.stack(symbol_boxes).reshape(-1,2,2), predictions)
axs[0].imshow(draw)
draw2 = draw_symbols(draw, thickness=8, color=(0,255,0))
axs[1].imshow(draw2)

In [ ]:
from mapcalc import calculate_map, calculate_map_range


gt_boxes = np.stack(data["symbols"]["box"])
gt_classes = np.stack(data["symbols"]["class"]).astype(int)
ground_truth_info={"boxes":gt_boxes,"labels":gt_classes}
pred_info={"boxes":symbol_boxes,"labels":predictions}#, "scores":confidences}

# calculate_map_range(ground_truth, predictions, 0.5,0.95,0.05)
calculate_map(ground_truth_info, pred_info, 0.5)